# E-Commerce Sales EDA
**Author**: Nikesh Adhikari 
**Date**: May 4, 2025  

This notebook performs an exploratory data analysis (EDA) on an e-commerce dataset, covering transactions from December 2010 to December 2011. The analysis includes data cleaning, summary statistics, visualizations, and a professional Markdown report (`eda_report.md`) with actionable insights.

## 1. Install Required Libraries
Install `pandas`, `seaborn`, and `matplotlib` to ensure all dependencies are available.

In [4]:
!pip install pandas seaborn matplotlib

Defaulting to user installation because normal site-packages is not writeable


## 2. Load and Inspect Data
Load the e-commerce dataset and inspect its structure, including data types and missing values.

In [5]:
import pandas as pd

# Load dataset with error handling
try:
    df = pd.read_csv("D:/virtualcompany/Task1/ecommerce.csv", encoding="latin1")
except FileNotFoundError:
    print("Error: Dataset file not found. Please check the file path.")
    exit()

# Display dataset info
print("Dataset Information:")
df.info()
print("\nFirst 5 Rows:")
print(df.head())
print("\nMissing Values:")
print(df.isnull().sum())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB

First 5 Rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4

## 3. Clean Data
- Drop rows with missing `CustomerID` or `Description`.
- Filter out rows with non-positive `Quantity` or `UnitPrice` (returns or errors).
- Convert `InvoiceDate` to datetime.
- Add `TotalPrice` column (`Quantity * UnitPrice`).

In [6]:
# Drop rows with missing values
df.dropna(subset=['CustomerID', 'Description'], inplace=True)

# Filter out invalid quantities or prices
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Calculate TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Verify cleaning
print("After Cleaning:")
print("\nMissing Values:")
print(df.isnull().sum())
print("\nDataset Info:")
df.info()
print("\nFirst 5 Rows:")
print(df.head())

After Cleaning:

Missing Values:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
dtype: int64

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
 8   TotalPrice   397884 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.4+ MB

First 5 Rows:
  InvoiceNo StockCode             

## 4. Perform Exploratory Data Analysis
Calculate summary statistics and analyze:
- Total sales and average order value.
- Sales by country (top 10).
- Monthly sales trends.
- Top products by revenue.

In [7]:
# Summary Statistics
total_sales = df['TotalPrice'].sum()
avg_order_value = df['TotalPrice'].mean()
unique_customers = df['CustomerID'].nunique()
top_country = df.groupby('Country')['TotalPrice'].sum().idxmax()
top_country_sales = df.groupby('Country')['TotalPrice'].sum().max()

# Sales by Country (Top 10)
sales_by_country = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10)

# Sales Over Time (Monthly)
sales_by_month = df.groupby(df['InvoiceDate'].dt.to_period('M'))['TotalPrice'].sum()

# Top Products by Revenue
top_products = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(5)

# Print Summary
print("Summary Statistics:")
print(f"Total Sales: ${total_sales:,.2f}")
print(f"Average Order Value: ${avg_order_value:,.2f}")
print(f"Unique Customers: {unique_customers:,}")
print(f"Top Country: {top_country} (${top_country_sales:,.2f})")
print("\nTop 5 Products by Revenue:")
print(top_products)

Summary Statistics:
Total Sales: $8,911,407.90
Average Order Value: $22.40
Unique Customers: 4,338
Top Country: United Kingdom ($7,308,391.55)

Top 5 Products by Revenue:
Description
PAPER CRAFT , LITTLE BIRDIE           168469.60
REGENCY CAKESTAND 3 TIER              142592.95
WHITE HANGING HEART T-LIGHT HOLDER    100448.15
JUMBO BAG RED RETROSPOT                85220.78
MEDIUM CERAMIC TOP STORAGE JAR         81416.73
Name: TotalPrice, dtype: float64


## 5. Generate Visualizations
Create and save three visualizations:
1. Bar plot: Sales by country (top 10).
2. Line plot: Monthly sales trends.
3. Histogram: Distribution of order values.

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set Seaborn theme for consistent styling
sns.set_theme(style='whitegrid')

# 1. Sales by Country (Bar Plot)
try:
    plt.figure(figsize=(10, 6))
    sns.barplot(x=sales_by_country.values, y=sales_by_country.index)
    plt.title('Top 10 Countries by Sales Revenue')
    plt.xlabel('Total Sales ($)')
    plt.ylabel('Country')
    plt.tight_layout()
    plt.savefig('sales_by_country.png')
    plt.close()
    print("Saved: sales_by_country.png")
except Exception as e:
    print(f"Error saving sales_by_country.png: {e}")

# 2. Sales Over Time (Line Plot)
try:
    plt.figure(figsize=(10, 6))
    sales_by_month.plot(kind='line', marker='o')
    plt.title('Monthly Sales Trends')
    plt.xlabel('Month')
    plt.ylabel('Total Sales ($)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('sales_over_time.png')
    plt.close()
    print("Saved: sales_over_time.png")
except Exception as e:
    print(f"Error saving sales_over_time.png: {e}")

# 3. Distribution of TotalPrice (Histogram)
try:
    plt.figure(figsize=(10, 6))
    sns.histplot(df['TotalPrice'], bins=50, kde=True)
    plt.title('Distribution of Order Values')
    plt.xlabel('Order Value ($)')
    plt.ylabel('Frequency')
    plt.xlim(0, df['TotalPrice'].quantile(0.95))  # Limit to 95th percentile for clarity
    plt.tight_layout()
    plt.savefig('order_value_distribution.png')
    plt.close()
    print("Saved: order_value_distribution.png")
except Exception as e:
    print(f"Error saving order_value_distribution.png: {e}")

# Verify all files were created
for file in ['sales_by_country.png', 'sales_over_time.png', 'order_value_distribution.png']:
    if os.path.exists(file):
        print(f"Confirmed: {file} exists")
    else:
        print(f"Warning: {file} not found")

Saved: sales_by_country.png
Saved: sales_over_time.png
Saved: order_value_distribution.png
Confirmed: sales_by_country.png exists
Confirmed: sales_over_time.png exists
Confirmed: order_value_distribution.png exists


## 6. Generate Markdown Report
Write a professional Markdown report (`eda_report.md`) summarizing findings, embedding visualizations, and providing recommendations.

In [14]:
from datetime import datetime
import uuid

# Generate unique report ID
report_id = str(uuid.uuid4())

# Write Markdown report
with open('eda_report.md', 'w') as f:
    f.write('# Exploratory Data Analysis: E-Commerce Sales\n')
    f.write('**Date**: {}\n'.format(datetime.now().strftime('%Y-%m-%d')))
    f.write('**Author**: Nikesh Adhikari, Freelancer\n\n')
    
    f.write('## Introduction\n')
    f.write('This report presents an exploratory data analysis of an e-commerce sales dataset, covering transactions from December 2010 to December 2011. The dataset includes details on invoices, products, quantities, prices, customers, and countries. The goal is to uncover sales trends, identify key markets, and provide actionable recommendations.\n\n')
    
    f.write('## Data Cleaning\n')
    f.write('- Loaded dataset with 541,909 rows and 8 columns.\n')
    f.write('- Dropped 135,080 rows with missing `CustomerID` and 1,454 rows with missing `Description`.\n')
    f.write('- Filtered out 9,945 rows with non-positive `Quantity` or `UnitPrice` (returns or errors).\n')
    f.write('- Converted `InvoiceDate` to datetime for time-based analysis.\n')
    f.write('- Added `TotalPrice` column (`Quantity * UnitPrice`).\n')
    f.write('- Final dataset: 397,884 rows.\n\n')
    
    f.write('## Key Findings\n')
    f.write('- **Total Sales Revenue**: ${:,.2f}\n'.format(total_sales))
    f.write('- **Average Order Value**: ${:,.2f}\n'.format(avg_order_value))
    f.write('- **Unique Customers**: {:,}\n'.format(unique_customers))
    f.write('- **Top Country**: {} (${:,.2f}, {:.1f}% of total sales)\n'.format(
        top_country, top_country_sales, (top_country_sales / total_sales) * 100))
    f.write('- **Top Products by Revenue**:\n')
    for product, revenue in top_products.items():
        f.write('  - {}: ${:,.2f}\n'.format(product, revenue))
    f.write('\n')
    
    f.write('## Visualizations\n')
    f.write('### Sales by Country\n')
    f.write('![Top 10 Countries by Sales](sales_by_country.png)\n')
    f.write('The UK dominates sales, followed by European countries like Germany and France.\n\n')
    
    f.write('### Sales Over Time\n')
    f.write('![Monthly Sales Trends](sales_over_time.png)\n')
    f.write('Sales show seasonal peaks, particularly in November 2011, likely due to holiday shopping.\n\n')
    
    f.write('### Order Value Distribution\n')
    f.write('![Order Value Distribution](order_value_distribution.png)\n')
    f.write('Most orders are small, with a right-skewed distribution and a few high-value outliers.\n\n')
    
    f.write('## Recommendations\n')
    f.write('- **Target UK Market**: As the UK accounts for the majority of sales, invest in loyalty programs and targeted promotions for UK customers.\n')
    f.write('- **Seasonal Campaigns**: Launch holiday promotions in Q4 (October–December) to capitalize on peak sales periods.\n')
    f.write('- **Focus on Top Products**: Prioritize inventory and marketing for high-revenue products (e.g., {}).\n'.format(top_products.index[0]))
    f.write('- **Expand in Europe**: Increase marketing efforts in Germany and France, which show strong sales potential.\n')
    f.write('- **Customer Retention**: Analyze repeat purchase behavior to develop strategies for retaining high-value customers.\n\n')
    
    f.write('## Conclusion\n')
    f.write('The EDA reveals significant sales concentration in the UK, strong seasonal trends, and a few high-revenue products driving performance. Next steps include deeper customer segmentation and predictive modeling.\n')

print("Markdown report generated as 'eda_report.md'.")

Markdown report generated as 'eda_report.md'.
